In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import matplotlib 
from PIL import Image
import PIL
import math
from random import random
from random import seed
from random import randint
import time
from numba import jit
import skimage

In [15]:
@jit(nopython = True)
    
def Laplacian(A,deltax,deltat):

    h = len(A)
    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    P = np.zeros((h,h))
    result = np.zeros((h,h))
    
    for i in range(0, h):
        for j in range(0,h):
            P[i,j] = (-A[(i+2)%h,j] + 16*A[(i+1)%h,j] + 16*A[(i-1)%h,j] -A[(i-2)%h,j] \
                 -A[i,(j+2)%h] + 16*A[i,(j+1)%h]  + 16*A[i,(j-1)%h] -A[i,(j-2)%h] - 60*A[i,j])/(12*pow(deltax,2))

  
    return P 

def damping(t,K_damp):
    Damping = np.zeros(shape=int(t/deltat))
    Damping[0:200] = 1*K_damp
    return Damping

def div_potential_1(A1,A2,omega,eta):
    lam = 2*pow(math.pi,2)/pow(omega,2)
    pot = lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2))
    return pot

def div_potential_2(A1,A2,omega,eta):
    lam = 2*pow(math.pi,2)/pow(omega,2)
    pot = lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2))
    return pot
    
def function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping):
    F = np.subtract(Laplacian(A1,deltax,deltat),div_potential_1(A1,A2,omega,eta)) - Damping[int(index)]*vel1
    return F

def function_2(A1,A2,vel2,omega,eta,deltax,deltat,index,Damping):
    F = np.subtract(Laplacian(A2,deltax,deltat),div_potential_2(A1,A2,omega,eta)) - Damping[int(index)]*vel2
    return F

def update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,index,Damping):
    F1 = function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping)
    F2 = function_2(A1,A2,vel2,omega,eta,deltax,deltat,index,Damping)
    
    B1 = A1
    
    A1 = A1 + deltat*(vel1 + 0.5*deltat*function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping)) 
    A2 = A2 + deltat*(vel2 + 0.5*deltat*function_2(B1,A2,vel2,omega,eta,deltax,deltat,index,Damping)) 
    
    vel1 = vel1 + deltat/2*(F1 + function_1(A1,A2,vel1,omega,eta,deltax,deltat,index,Damping)) 
    vel2 = vel2 + deltat/2*(F2 + function_2(A1,A2,vel2,omega,eta,deltax,deltat,index,Damping))
    return A1,A2,vel1,vel2
    
#change the values accordlingly    
def wolff(N,t,A1,A2,vel,omega,eta,deltax,deltat,K_damp):

    Damping = damping(t,K_damp)

    vel1 = vel
    vel2 = vel

    y = []
    

    for i in np.arange(int(t/deltat)):
        
        w = []
        
        #plt.imsave('ComplexIM/'+str(round((i*deltat),1))+'.png',np.sqrt(pow(A1,2)+pow(A2,2)),vmin = 0, vmax = 1.5*eta,cmap = 'Greys')
        #plt.imsave('ComplexIM/'+str(round((i*deltat),1))+'a.png',np.arctan2(A2,A1),vmin = -1*math.pi, vmax = math.pi,cmap = 'twilight')
        A1,A2,vel1,vel2 = update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,i,Damping)
        z = np.arctan2(A2,A1)
        x = z.tolist()
        w.extend(x)
        y.append(np.median(w))
    return A1,A2,y


In [20]:
start = time.time()

K_damp = 1
n = 7
deltax = 1
deltat = 0.1
eta = 1
N = pow(2,n)
t = 0.5*N*deltax/deltat # light crossing time
t2 = t/2 # half time
t10 = t/10 # 1/10 of the ligth crossing time
vel = np.zeros((N,N))
mu, sigma = 0, 0.1 # mean and standard deviation
omega = 5


#A = np.random.randn(N, 2*N).view(np.complex128)   
avg = []
for m in range(20):
    A1 = np.random.normal(mu, sigma, size=(N, N))
    A2 = np.random.normal(mu,sigma,size=(N,N))
    A1,A2,y = wolff(N,t10,A1,A2,vel,omega,eta,deltax,deltat,K_damp)
    avg.append(sum(y)/len(y))

print(avg)
print(sum(avg)/len(avg))
end = time.time()
print(end - start)

[-0.7245929265924695, -0.5848757086546115, -0.9555085952895588, 0.318273779402323, -0.7316926946173103, -0.7485026959507116, -0.20558522678064342, -0.5642630044534294, -0.6776390074198568, 0.4834360752536787, -0.7526024153484109, -0.9319530389588531, 0.4486442366608198, 0.14898959014008337, -0.5087203020141537, -0.1628930597355568, -0.32523344028698703, -0.8000826680391053, -0.32877028231734834, 0.16483469843174073, 0.4373888571196525, 0.33222807841382035, 0.8684467281817655, -0.312949513231607, -0.3241635553206637, 0.021362648463158378, -0.585032553223536, -0.14745589967169198, -0.6899427840022285, 0.07069419600599496, 0.014325776218836638, 0.08048381488755972, 0.4761774674293796, -0.6849366265232568, 0.2452726758472243, 0.1421081506643497, 0.12888608841190757, -0.23200464735399667, -0.004098593936617051, -0.23208948923776823, 0.5183734504754132, 0.37683167293051045, -0.6796001212349613, -0.09177086321181321, -0.3055061612163258, -0.6379536666015917, -0.24428676045039235, -0.090284271

AttributeError: 'list' object has no attribute 'mean'

In [8]:
Laplacian(np.zeros((10,10)),1,0.1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [21]:
print(sum(avg)/len(avg))

-0.19407686702826435
